# TP3: Arithmétique et cryptographie

## 1. Arithmétique modulaire

1. Construisez la table d'addition et de multiplication des entiers modulo 5. Pour une présentation plus agréable, considérez la fonction `Matrix` comme dans le TP 2

In [1]:
from sympy import * 
#init_printing()
x,y = symbols('x y')
add = Lambda([x,y], ((x+y)%5))  # pour l'addition
Matrix(5,5,add)

Matrix([
[0, 1, 2, 3, 4],
[1, 2, 3, 4, 0],
[2, 3, 4, 0, 1],
[3, 4, 0, 1, 2],
[4, 0, 1, 2, 3]])

In [2]:
mul = Lambda([x,y], ((x*y)%5)) # pour la multiplication
Matrix(5,5,mul)

Matrix([
[0, 0, 0, 0, 0],
[0, 1, 2, 3, 4],
[0, 2, 4, 1, 3],
[0, 3, 1, 4, 2],
[0, 4, 3, 2, 1]])

2. Ecrivez une fonction `CoprimeQ` qui prend en entrée deux entiers et qui renvoie `True` si les entiers sont premiers entre eux

In [3]:
def CoprimeQ(p,q):
    return gcd(p,q) == 1
CoprimeQ(3,11)

True

3. Programmez l'algorithme d'Euclide étendu itératif suivant et vérifiez qu'il fonctionne bien en comparant avec celui de la librairie `Sympy`. La fonction `floor` se trouve dans la librairie `math` mais corresond aussi à la division entière `//`. Les variables `Q` et `R` ne peuvent pas être des tuples (il n'y a pas d'arithmétique des tuples).

In [4]:
def euclideEtendu(a,b):
    d,u,v,d1,u1,v1=a,1,0,b,0,1
    while d1!=0:
        q=d//d1
        d,u,v,d1,u1,v1=d1,u1,v1,d-q*d1,u-q*u1,v-q*v1
    return (u,v,d)

euclideEtendu(11,26)

(-7, 3, 1)

In [5]:
gcdex(3,26)

(9, -1, 1)

Pour comprendre le fonctionnement, remplissez le tableau qui trace les valeurs de `q,r,t,Q,floor(q/r),R,T`. ![](EucEt1126.png)

4. Ecrivez une fonction `modinv` qui prend en entrée `a` et `m` et qui renvoie l'inverse de `a`modulo `m` s'il existe et retourne une erreur sinon

In [6]:
def modinv(a,m):
    invmod = gcdex(a,m)
    if invmod[2] !=1 : return -1
    return invmod[0]%m
modinv(3,26)

9

5. Reprenez le code de l'exponentielle modulaire $a^b\mod n$ et instrumentez le code pour faire la trace d'exécution en remplissant un tableau comme dans l'exemple ![](exponentMod.png)

In [ ]:
def exponentMod(a,b,p):
    res=1
    for i in bin(b)[2:]:
        res=(res*res)%p
        if i=='1':
            res=(res*a)%p
        print(i,res)
    return(res)

## 2. RSA

1. Ecrivez une fonction `genCle` qui prend en entrée une taille (celle de la clé) exprimée en octets (1 octet = 8 bits) et qui retourne le tuple composé de l'entier $p$ et de l'entier $q$.

In [7]:
def genCle(n):
    gen = randprime(1,n*2**8)
    return (gen,nextprime(gen))
genCle(4)

(317, 331)

2. Ecrivez les fonctions `clePub` et `clePriv` qui ont un fonctionnement analogue à `rsa_public_key` et `rsa_private_key`. N'oubliez pas de fournir la valeur de `e`.

In [8]:
def clePub(p,q,e):
    n = p*q
    if gcd(e,totient(n)) != 1 : return False
    return (n,e)

def clePriv(p,q,e):
    n = p*q
    if modinv(e,totient(n)) == -1 : return False
    return modinv(e,totient(n))
clePriv(nextprime(100),nextprime(101),7)

8743

3. Ecrivez une fonction `RSA`en `Python` qui prend en entrée :
    - soit la clé publique: le tuple $(n,e)$ et un entier $m$ et qui renvoie $m^e\mod n$
    - soit la clé privée: le tuple $(n,d)$ et un entier $c$ et qui renvoie $c^d\mod n$

In [9]:
def rsa(m,t):
    return (m**t[1])%t[0]
rsa(3,(509,11))

15

In [10]:
rsa(3,(509,11))

15

4. Utilisez votre fonction `RSA` pour construire `RSAEnc` et `RDADec` dont le comportement est analogue à `encipher_rsa(m,pk)` et `decipher_rsa(c,dk)` de l'implémentation RSA de `Sympy`.

In [11]:
def RSAEnc(m,pk):
    return pow(m,pk[1],pk[0])
RSAEnc(3,(509*521,11))

177147

In [13]:
def RSADec(c,dk):
    return pow(c,dk[1],dk[0])
RSADec(177147,(509*521,216131))

3

5. En utilisant les fonctions de codage et de décodage du cours, tarnsformez les fonctions `RSAEnc` et `RSADec` pour qu'elles pemettent de chiffrer et de déchiffer des chaines de caractères.

In [19]:
DEFAULT_BLOCK_SIZE = 128
BYTE_SIZE=256

def Text2Ints(message,blockSize=DEFAULT_BLOCK_SIZE):
    # convertit une chaine en une liste de blocs d'entiers
    # chaque entier représente 'blockSize' caractères de la chaîne
            
    messageBytes=message.encode('ascii')   # convertit la chaine en octets 
    blockInts=[]
    for blockStart in range(0,len(messageBytes),blockSize):
        # calculer l'entier correspondant au bloc de texte
        blockInt=0
        for i in range(blockStart,min(blockStart+blockSize,len(messageBytes))):
             blockInt += int(messageBytes[i])*(BYTE_SIZE ** (i%blockSize))
        blockInts.append(blockInt)
        
    return blockInts


Text2Ints("ceci est un message",2)


[25955, 26979, 25888, 29811, 29984, 8302, 25965, 29555, 26465, 101]

In [23]:
def RSAEnc1(m,pk,blockSize=DEFAULT_BLOCK_SIZE):
    if type(m) == str :
        m = Text2Ints(m,blockSize)
        return [pow(i,pk[1],pk[0]) for i in m]
    return pow(m,pk[1],pk[0])

In [24]:
RSAEnc1("ceci est un message",(10403,7),2)

[9063, 4886, 3473, 4928, 8944, 7002, 1503, 7331, 2490, 6464]

In [26]:
def Ints2Text(blockInts,messageLength,blockSize=DEFAULT_BLOCK_SIZE) :
    message=[]
    for blockInt in blockInt:
        blockMessage=[]
        for i in range(blockSize-1,-1,-1):
            if len(message)+i < messageLength :
                charNumber=blockInt//(BYTE_SIZE**i)
                blockInt=blockInt%(BYTE_SIZE**i)
                blockMessage.insert(0,bytes([charNumber]).decode('ascii'))
        message.extend(blockMessage)
    return ''.join(message)

In [25]:
def RSADec2(c,dk):
    if type(c)==list :
        return [pow(i,dk[1],dk[0]) for i in c]
    return pow(c,dk[1],dk[0])
RSADec2([9063, 4886, 3473, 4928, 8944, 7002, 1503, 7331, 2490, 6464],(10403,8743))

[5149, 6173, 5082, 9005, 9178, 8302, 5159, 8749, 5659, 101]